In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
tbasics_file = "/kaggle/input/imdb-basic-dataset/title.basics.tsv/data.tsv"
eposode_file = "/kaggle/input/imdb-basic-dataset/title.episode.tsv/data.tsv"
# Big file
principals_file = "/kaggle/input/imdb-basic-dataset/title.principals.tsv/data.tsv"
ratings_file = "/kaggle/input/imdb-basic-dataset/title.ratings.tsv/data.tsv"
nbasics_file = "/kaggle/input/imdb-basic-dataset/name.basics.tsv/data.tsv"
# very big file
akas_file = "/kaggle/input/imdb-basic-dataset/title.akas.tsv/data.tsv"

crew_file = "/kaggle/input/imdb-basic-dataset/title.crew.tsv/data.tsv"

In [3]:
basics = pd.read_csv(tbasics_file, sep='\t')


/tmp/ipykernel_20/1105777491.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv(tbasics_file, sep='\t')


In [4]:
ratings = pd.read_csv(ratings_file, sep='\t')

tbasics = basics.join(ratings.set_index('tconst'), on="tconst")

In [5]:
tbasics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1980.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,265.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1835.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,179.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2624.0
...,...,...,...,...,...,...,...,...,...,...,...
9928243,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family",NaN,NaN
9928244,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family",NaN,NaN
9928245,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family",NaN,NaN
9928246,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short,NaN,NaN


In [6]:
tbasics.groupby(['titleType']).titleType.count()

titleType
movie            648082
short            935682
tvEpisode       7546405
tvMiniSeries      48929
tvMovie          142038
tvPilot               1
tvSeries         244950
tvShort            9968
tvSpecial         41988
video            275322
videoGame         34883
Name: titleType, dtype: int64

# join the episode to the series

I use pandas to shape and transform my data for TV Shows into something a little more helpful

In [7]:
tvSeries = tbasics.loc[(tbasics['titleType'] == "tvSeries")].rename(columns={"tconst":"parentTconst", "primaryTitle":"TVShow"})

In [8]:
tvEpisode = tbasics.loc[(tbasics['titleType'] == "tvEpisode")].rename(columns={"primaryTitle": "eposodeTitle", "originalTitle": "originalEposode", "startYear": "year", "runtimeMinutes":"minutes"})

In [9]:
eposodes = pd.read_csv(eposode_file, sep='\t').rename(columns={"seasonNumber": "S", "episodeNumber": "E"})

In [10]:
TV = eposodes.join(tvSeries.drop([
        'isAdult','runtimeMinutes','genres','titleType','averageRating', 'numVotes'
    ], axis=1).set_index('parentTconst'), on="parentTconst").join(tvEpisode.drop(['titleType',"endYear"], axis=1).set_index('tconst'), on="tconst").drop(['parentTconst'],axis=1)

In [11]:
TV

,tconst,S,E,TVShow,originalTitle,startYear,endYear,eposodeTitle,originalEposode,isAdult,year,minutes,genres,averageRating,numVotes
0,tt0041951,1,9,The Lone Ranger,The Lone Ranger,1949,1957,The Tenderfeet,The Tenderfeet,0,1949,30,Western,7.5,81.0
1,tt0042816,1,17,BBC Sunday-Night Theatre,BBC Sunday-Night Theatre,1950,1959,Othello,Othello,0,1950,135,Drama,NaN,NaN
2,tt0042889,\N,\N,BBC Sunday-Night Theatre,BBC Sunday-Night Theatre,1950,1959,The Tragedy of King Richard II/II,The Tragedy of King Richard II/II,0,1950,145,Drama,NaN,NaN
3,tt0043426,3,42,Studio One,Studio One,1948,1958,Coriolanus,Coriolanus,0,1951,60,Drama,NaN,NaN
4,tt0043631,2,16,BBC Sunday-Night Theatre,BBC Sunday-Night Theatre,1950,1959,The Life of King Henry V,The Life of King Henry V,0,1951,133,Drama,6.8,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7546305,tt9916846,3,18,Arka Siradakiler,Arka Siradakiler,2007,2012,Episode #3.18,Episode #3.18,0,2010,\N,"Action,Drama,Family",NaN,NaN
7546306,tt9916848,3,17,Arka Siradakiler,Arka Siradakiler,2007,2012,Episode #3.17,Episode #3.17,0,2010,\N,"Action,Drama,Family",NaN,NaN
7546307,tt9916850,3,19,Arka Siradakiler,Arka Siradakiler,2007,2012,Episode #3.19,Episode #3.19,0,2010,\N,"Action,Drama,Family",NaN,NaN
7546308,tt9916852,3,20,Arka Siradakiler,Arka Siradakiler,2007,2012,Episode #3.20,Episode #3.20,0,2010,\N,"Action,Drama,Family",NaN,NaN


# here is a list of videos not tv

This is just a little hack.  not all entries in title.episode matchs every 'tvEpisode' titleType in title.basics.

I will need to look into dropping the rows with ```tconst``` values match the ones in TV.

In [12]:
Videos = tbasics.loc[(tbasics['titleType'] != "tvSeries") & (tbasics['titleType'] != "tvEpisode")].rename(columns={"startYear": "year", "runtimeMinutes":"minutes"}).drop(["endYear"], axis=1)
Videos

,tconst,titleType,primaryTitle,originalTitle,isAdult,year,minutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,1,"Documentary,Short",5.7,1980.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,5,"Animation,Short",5.8,265.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,4,"Animation,Comedy,Romance",6.5,1835.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,12,"Animation,Short",5.6,179.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,1,"Comedy,Short",6.2,2624.0
...,...,...,...,...,...,...,...,...,...,...
9928188,tt9916730,movie,6 Gunn,6 Gunn,0,2017,116,\N,8.3,10.0
9928198,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013,49,Documentary,NaN,NaN
9928199,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,0,2019,\N,Short,NaN,NaN
9928203,tt9916764,short,38,38,0,2018,\N,Short,NaN,NaN


In [13]:
#tvMiniSeries = tbasics.loc[(tbasics['titleType'] == "tvMiniSeries")]
#tvSpecial = tbasics.loc[(tbasics['titleType'] == "tvSpecial")]
#tvPilot = tbasics.loc[(tbasics['titleType'] == "tvPilot")]
#video = tbasics.loc[(tbasics['titleType'] == "video")]
#tvPilot = tbasics.loc[(tbasics['titleType'] == "videoGame")]

In [14]:
#movies
#tvMovie = tbasics.loc[(tbasics['titleType'] == "tvMovie")]
#movie = tbasics.loc[(tbasics['titleType'] == "movie")]

In [15]:
#shorts
#tvShort = tbasics.loc[(tbasics['titleType'] == "tvShort")]
#short = tbasics.loc[(tbasics['titleType'] == "short")]